In [ ]:
# following nidaqmx tutorial on the nspyre documention

In [1]:
import nidaqmx
from nidaqmx.constants import (AcquisitionType, CountDirection, Edge,
    READ_ALL_AVAILABLE, TaskMode, TriggerType)
from nidaqmx.stream_readers import CounterReader
import numpy as np

In [2]:
system = nidaqmx.system.System.local()

In [3]:
system.devices[0]

Device(name=Dev1)

In [4]:
DAQ_device = system.devices['Dev1']
# create a list of all the counters available on 'DAQ1'
counter_names = [ci.name for ci in DAQ_device.ci_physical_chans]
print(counter_names)
# note that using the counter output channels instead of the inputs
# includes the '[device]/freqout' output, which is not a counter
print([co.name for co in DAQ_device.co_physical_chans])

['Dev1/ctr0', 'Dev1/ctr1', 'Dev1/ctr2', 'Dev1/ctr3']
['Dev1/ctr0', 'Dev1/ctr1', 'Dev1/ctr2', 'Dev1/ctr3', 'Dev1/freqout']


In [5]:
counter_names = [ci.name for ci in DAQ_device.ai_physical_chans]
print(counter_names)
# note that using the counter output channels instead of the inputs
# includes the '[device]/freqout' output, which is not a counter
print([co.name for co in DAQ_device.ao_physical_chans])

['Dev1/ai0', 'Dev1/ai1', 'Dev1/ai2', 'Dev1/ai3', 'Dev1/ai4', 'Dev1/ai5', 'Dev1/ai6', 'Dev1/ai7', 'Dev1/ai8', 'Dev1/ai9', 'Dev1/ai10', 'Dev1/ai11', 'Dev1/ai12', 'Dev1/ai13', 'Dev1/ai14', 'Dev1/ai15', 'Dev1/ai16', 'Dev1/ai17', 'Dev1/ai18', 'Dev1/ai19', 'Dev1/ai20', 'Dev1/ai21', 'Dev1/ai22', 'Dev1/ai23', 'Dev1/ai24', 'Dev1/ai25', 'Dev1/ai26', 'Dev1/ai27', 'Dev1/ai28', 'Dev1/ai29', 'Dev1/ai30', 'Dev1/ai31']
['Dev1/ao0', 'Dev1/ao1', 'Dev1/ao2', 'Dev1/ao3']


In [6]:
print([d for d in DAQ_device.di_ports])

[PhysicalChannel(name=Dev1/port0), PhysicalChannel(name=Dev1/port1), PhysicalChannel(name=Dev1/port2)]


In [7]:
counter_names = [ci for ci in DAQ_device.terminals]
print(counter_names)


['/Dev1/PFI0', '/Dev1/PFI1', '/Dev1/PFI2', '/Dev1/PFI3', '/Dev1/PFI4', '/Dev1/PFI5', '/Dev1/PFI6', '/Dev1/PFI7', '/Dev1/PFI8', '/Dev1/PFI9', '/Dev1/PFI10', '/Dev1/PFI11', '/Dev1/PFI12', '/Dev1/PFI13', '/Dev1/PFI14', '/Dev1/PFI15', '/Dev1/APFI0', '/Dev1/APFI1', '/Dev1/RTSI0', '/Dev1/RTSI1', '/Dev1/RTSI2', '/Dev1/RTSI3', '/Dev1/RTSI4', '/Dev1/RTSI5', '/Dev1/RTSI6', '/Dev1/RTSI7', '/Dev1/20MHzTimebase', '/Dev1/100MHzTimebase', '/Dev1/10MHzRefClock', '/Dev1/ai/ReferenceTrigger', '/Dev1/ai/SampleClock', '/Dev1/ai/StartTrigger', '/Dev1/ao/SampleClock', '/Dev1/ao/StartTrigger', '/Dev1/di/SampleClock', '/Dev1/di/StartTrigger', '/Dev1/di/ReferenceTrigger', '/Dev1/do/SampleClock', '/Dev1/do/StartTrigger', '/Dev1/ChangeDetectionEvent', '/Dev1/WatchdogExpiredEvent', '/Dev1/WatchdogExpirationTrigger', '/Dev1/AnalogComparisonEvent', '/Dev1/ai/ConvertClock', '/Dev1/ai/ConvertClockTimebase', '/Dev1/ai/HoldCompleteEvent', '/Dev1/ai/PauseTrigger', '/Dev1/ai/SampleClockTimebase', '/Dev1/ao/PauseTrigger',

In [8]:
read_task = nidaqmx.Task()
samp_clk_task = nidaqmx.Task()

In [9]:
dev_name = 'Dev1'
clock_di_cannel = 'port0'

In [10]:
samp_clk_task.di_channels.add_di_chan(f'{dev_name}/{clock_di_cannel}')

DIChannel(name=Dev1/port0)

In [11]:
sampling_rate = 1e2
samp_clk_task.timing.cfg_samp_clk_timing(sampling_rate,
                                sample_mode=AcquisitionType.CONTINUOUS)

In [12]:
samp_clk_task.control(TaskMode.TASK_COMMIT)


In [13]:
read_task.ci_channels.add_ci_count_edges_chan(
                                f'{dev_name}/ctr0',
                                edge=Edge.RISING,
                                initial_count=0,
                                count_direction=CountDirection.COUNT_UP)

CIChannel(name=Dev1/ctr0)

In [14]:
read_task.ci_channels.all.ci_count_edges_term = f'/{dev_name}/PFI0'

In [15]:
read_task.timing.cfg_samp_clk_timing(sampling_rate, source=f'/{dev_name}/di/SampleClock',
        active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)

In [16]:
read_task.in_stream.input_buf_size = 12000

In [17]:
read_task.triggers.arm_start_trigger.trig_type = TriggerType.DIGITAL_EDGE
read_task.triggers.arm_start_trigger.dig_edge_edge = Edge.RISING
read_task.triggers.arm_start_trigger.dig_edge_src = f'/{dev_name}/di/SampleClock'

In [18]:
samp_clk_task.stop()
read_task.stop()


In [19]:
# create a counter reader to read from the counter InStream
reader = CounterReader(read_task.in_stream)
# start the tasks to begin data acquisition; note that because
# the arm start trigger of the counter was set, it does not
# matter which task is started first, the tasks will be synced
samp_clk_task.start()
read_task.start()
# create a data buffer for the counter stream reader
data_array = np.zeros(12000, dtype=np.uint32)
# read all samples from the counter buffer to the system memory
# buffer data_array; if the buffer is not large enough, it will
# raise an error
reader.read_many_sample_uint32(data_array,
    number_of_samples_per_channel=READ_ALL_AVAILABLE)

DaqError: Read cannot be performed because the NumPy array passed into this function is not shaped correctly. You must pass in a NumPy array of the correct shape based on the number of channels in task and the number of samples per channel requested.

Shape of NumPy Array provided: (12000,)
Shape of NumPy Array required: (0,)

Task Name: _unnamedTask<0>